# Introduction
- **Nama:** NANTHA SEUTIA
- **Email:** ombakrinai@gmail.com



# Import Library

In [ ]:
!pip install pymupdf nltk sentence-transformers langchain scikit-learn

import os
import re
import json
import fitz # PyMuPDF
import nltk
import requests
import numpy as np
import logging # Tambahkan logging
import time # Untuk mengukur waktu

from nltk import sent_tokenize
from sentence_transformers import SentenceTransformer, CrossEncoder, util # Import util untuk semantic_search
from langchain.text_splitter import RecursiveCharacterTextSplitter # Import splitter baru

# Download resource NLTK
try:
    nltk.data.find('tokenizers/punkt')
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    print("Downloading NLTK 'punkt'...")
    nltk.download('punkt')
    nltk.download('punkt_tab')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


# Data Gathering

In [ ]:
# ===============================
# 1. DATA GATHERING
# ===============================
logging.info("Starting Data Gathering...")
GITHUB_RAW_URL = "https://raw.githubusercontent.com/esnanta/ai-chatbot-dana-bos-api/main/knowledge_base/"
FILES = [
    "Permendikbudriset_No_63_Tahun_2023.pdf",
]

pdf_dir = "/content/pdf_files" # Direktori penyimpanan di Colab
os.makedirs(pdf_dir, exist_ok=True)
for file in FILES:
    file_url = GITHUB_RAW_URL + file
    try:
        response = requests.get(file_url)
        response.raise_for_status()
        with open(os.path.join(pdf_dir, file), "wb") as f: f.write(response.content)
        logging.info(f"✅ Downloaded: {file}")
    except requests.exceptions.RequestException as e:
        logging.error(f"❌ Download failed for {file}: {e}")
    except Exception as e:
        logging.error(f"❌ Unexpected error downloading {file}: {e}")
logging.info(f"Files in {pdf_dir}: {os.listdir(pdf_dir)}")

In [ ]:
# ===============================
# 2. EKSTRAKSI TEKS DARI FILE PDF
# ===============================

# --- PDF Text Extraction ---
def extract_text_from_pdf(pdf_path):
    """Extracts text from a PDF file."""
    try:
        with fitz.open(pdf_path) as doc:  # Use context manager for safety
            text = ""
            for page in doc:
                text += page.get_text("text") + "\n"
        return text.strip()
    except Exception as e:
        raise RuntimeError(f"Gagal mengekstrak teks dari {pdf_path}: {e}")


pdf_files = [f for f in os.listdir(pdf_dir) if f.endswith(".pdf")]
pdf_texts = {}

for pdf_file in pdf_files:
    pdf_path = os.path.join(pdf_dir, pdf_file)
    try:
        text = extract_text_from_pdf(pdf_path)
        pdf_texts[pdf_file] = text
        print(f"✅ Berhasil mengekstrak teks dari: {pdf_file}")
    except RuntimeError as re:
        print(f"❌ Kesalahan saat ekstraksi: {re}")
    except Exception as e:
        print(f"❌ Kesalahan tidak terduga pada {pdf_file}: {e}")


# Cek hasil ekstraksi
print(f"\nTotal file yang berhasil diekstrak: {len(pdf_texts)}")

✅ Berhasil mengekstrak teks dari: Permendikbudriset_No_63_Tahun_2023.pdf

Total file yang berhasil diekstrak: 1


# Preprocessing Data

In [ ]:
# ===============================
# 3. PREPROCESSING TEKS
# ===============================
def clean_text(text):
    # This collapses multiple consecutive blank lines into a single blank line,
    # reducing unnecessary whitespace.
    text = re.sub(r'\n+', '\n', text) # Ganti \n+ -> \n dulu
    text = re.sub(r'\s+', ' ', text).strip() # Baru ganti whitespace -> spasi

    # Pasal 17. -> Pasal 17 (spasi)
    text = re.sub(r'Pasal (\d+)\.(?=\s|$)', r'Pasal \1 ', text) # Lebih robust: . diikuti spasi atau akhir string

    # Ayat (1a). -> Ayat (1a) (spasi)
    text = re.sub(r'Ayat \((\d+[a-z]?)\)\.(?=\s|$)', r'Ayat (\1) ', text)

    text = re.sub(r'http\S+|www\S+', '', text, flags=re.IGNORECASE)  # Remove URLs
    text = re.sub(r'jdih\.kemdikbud\.go\.id', '', text, flags=re.IGNORECASE)  # Remove specific website

    # Replace page number pattern '- 4 -' with '(page 4)'
    text = re.sub(r'\s-\s(\d+)\s-\s', r' (page \1) ', text) # Tetap berguna

    # Hapus pola "angka titik" di akhir kalimat yang mungkin terlewat
    # contoh: "kalimat selesai 1." -> "kalimat selesai"
    text = re.sub(r'(\w)\s+(\d+)\.\s*$', r'\1', text)

    return text

print("Cleaning texts...")
cleaned_texts = {pdf: clean_text(text) for pdf, text in pdf_texts.items()}
print(f"Cleaned {len(cleaned_texts)} documents.")

Cleaning texts...
Cleaned 1 documents.


In [ ]:
# ===============================
# 4. CHUNKING TEKS (Menggunakan RecursiveCharacterTextSplitter)
# ===============================
print("Chunking texts...")

# Inisialisasi Text Splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,      # Ukuran chunk dalam karakter (eksperimen dengan nilai ini)
    chunk_overlap=150,    # Jumlah overlap antar chunk (eksperimen)
    length_function=len,
    is_separator_regex=False,
    separators=["\n\n", "\n", ". ", "? ", "! ", " ", ""], # Pemisah prioritas
)

# --- Fungsi tambahan untuk post-processing chunk (opsional tapi disarankan) ---
def post_process_chunk(chunk):
    """Membersihkan chunk setelah splitting."""
    # Hapus angka/bullet/simbol di awal chunk (setelah splitting mungkin muncul lagi)
    chunk = re.sub(r'^\s*[\d\-\•\*\.]+\s*', '', chunk)
    # Hapus pola nomor halaman yang mungkin masih ada di tengah
    chunk = re.sub(r'\s*\(page \d+\)\s*', ' ', chunk)
     # Hapus pola pasal/ayat yang mungkin berdiri sendiri tanpa konteks berarti setelah split
    chunk = re.sub(r'^\s*(Pasal \d+[a-z]?|Ayat \(\d+[a-z]?\))\s*$', '', chunk, flags=re.IGNORECASE)
    # Hilangkan spasi berlebihan yang mungkin muncul lagi
    chunk = re.sub(r'\s+', ' ', chunk).strip()
    return chunk

# --- Fungsi filter chunk yang terlalu pendek atau tidak relevan ---
def filter_short_or_irrelevant(chunk, min_length=50):
    """Menghapus chunk yang terlalu pendek atau hanya berisi noise."""
    if len(chunk) < min_length:
        return False
    # Contoh pola tidak relevan (bisa ditambahkan sesuai kebutuhan)
    if re.match(r'^\s*(daftar isi|kata pengantar|lampiran)\s*$', chunk, re.IGNORECASE):
        return False
    # Cek apakah chunk hanya berisi spasi atau karakter non-alfanumerik
    if not re.search(r'[a-zA-Z0-9]', chunk):
         return False
    return True

# Proses chunking untuk semua teks
all_raw_chunks = []
for pdf, text in cleaned_texts.items():
    # Gunakan text_splitter untuk memecah teks
    chunks = text_splitter.split_text(text)
    all_raw_chunks.extend(chunks)

# Post-process dan filter chunks
processed_chunks = [post_process_chunk(chunk) for chunk in all_raw_chunks]
final_chunks = [chunk for chunk in processed_chunks if filter_short_or_irrelevant(chunk)]

# Ganti nama variabel agar konsisten
all_chunks = final_chunks # final_chunks sekarang menjadi data utama kita

print(f"Total final chunks after processing and filtering: {len(all_chunks)}")
# Optional: Tampilkan beberapa contoh chunk
# for i, chunk in enumerate(all_chunks[:3]):
#      print(f"Chunk {i+1}:\n{chunk}\n-------")

Chunking texts...
Total final chunks after processing and filtering: 83


# SAVING CHUNK FILE

In [ ]:
# ===============================
# 5. SAVING CHUNKS
# ===============================
chunks_file = os.path.join(pdf_dir, "chunks.json")

print(f"Saving {len(all_chunks)} final chunks...")
try:
    with open(chunks_file, "w", encoding="utf-8") as f:
        json.dump(all_chunks, f, ensure_ascii=False, indent=4)
    print(f"Final chunks saved to: {chunks_file}")
except Exception as e:
    print(f"Error saving final chunks: {e}")

Saving 83 final chunks...
Final chunks saved to: /content/pdf_files/chunks.json


# LOAD MODEL

In [ ]:
# ===============================
# 6. MODEL LOADING & ENCODING (Embedder + CrossEncoder, TANPA FAISS)
# ===============================
logging.info("Loading Embedding and Reranking models...")
# --- Pilih Model ---
# Embedder (untuk retrieval awal)
# Pastikan embeddings (jika dimuat dari file) cocok dengan model ini!
# embedder_model_name = "paraphrase-MiniLM-L3-v2" # Lebih kecil, mungkin kurang akurat
# embedder_model_name = "all-MiniLM-L6-v2"         # Pilihan umum yang baik
embedder_model_name = "multi-qa-MiniLM-L6-cos-v1" # Baik untuk Q&A

# CrossEncoder (untuk reranking)
# cross_encoder_model_name = "cross-encoder/ms-marco-TinyBERT-L-6" # Sangat kecil
# cross_encoder_model_name = "cross-encoder/ms-marco-MiniLM-L-6-v2" # Pilihan umum yang baik
cross_encoder_model_name = "cross-encoder/ms-marco-MiniLM-L-12-v2" # Lebih besar, mungkin lebih akurat

logging.info(f"Using Embedder: {embedder_model_name}")
logging.info(f"Using CrossEncoder: {cross_encoder_model_name}")
logging.warning("Ensure sufficient RAM is available for both models.")

try:
    embedder = SentenceTransformer(embedder_model_name)
    cross_encoder = CrossEncoder(cross_encoder_model_name) # Muat CrossEncoder
except Exception as e:
    logging.error(f"Error loading models: {e}")
    raise

# --- Encoding Chunks ---
# Cek jika chunk_embeddings sudah ada atau perlu dibuat
embedding_file = os.path.join(pdf_dir, "chunk_embeddings.npy")
if os.path.exists(embedding_file):
    logging.info(f"Loading pre-computed embeddings from {embedding_file}")
    chunk_embeddings = np.load(embedding_file)
    logging.info(f"Loaded embeddings with shape {chunk_embeddings.shape}")
    # Validasi dimensi
    expected_dim = embedder.get_sentence_embedding_dimension()
    if chunk_embeddings.shape[1] != expected_dim:
         raise ValueError(f"CRITICAL ERROR: Loaded embedding dimension ({chunk_embeddings.shape[1]}) != model dimension ({expected_dim}). Regenerate embeddings with '{embedder_model_name}' or load correct file.")
    # Validasi jumlah
    if len(all_chunks) != chunk_embeddings.shape[0]:
         logging.warning(f"Mismatch chunks ({len(all_chunks)}) vs embeddings ({chunk_embeddings.shape[0]}). Ensure chunks.json and embeddings.npy match.")
         # Anda mungkin perlu memuat ulang all_chunks dari file json jika ada ketidaksesuaian
else:
    logging.info(f"No pre-computed embeddings found. Encoding {len(all_chunks)} chunks...")
    if not all_chunks:
        raise ValueError("Cannot encode embeddings: 'all_chunks' list is empty.")
    chunk_embeddings = embedder.encode(all_chunks, convert_to_numpy=True, show_progress_bar=True)
    logging.info(f"Encoded {chunk_embeddings.shape[0]} chunks with dimension {chunk_embeddings.shape[1]}.")
    # Simpan embeddings yang baru dibuat
    logging.info(f"Saving newly created embeddings to {embedding_file}")
    try:
        np.save(embedding_file, chunk_embeddings)
        logging.info(f"Embeddings saved successfully.")
    except Exception as e:
        logging.error(f"Error saving embeddings: {e}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

# Answer

In [ ]:
# ===============================
# 7. FUNGSI UTAMA Q&A (Hanya Embedder + Cosine Similarity)
# ===============================

# Hapus fungsi extract_keywords dan filter_chunks_by_keywords

def answer_question_with_rerank(
    question: str,
    chunks: list,
    embeddings: np.ndarray,
    embedder_model: SentenceTransformer,
    cross_encoder_model: CrossEncoder,
    top_k_retrieval: int = 10, # Jumlah kandidat awal untuk diambil
    top_n_final: int = 3       # Jumlah jawaban akhir setelah reranking
) -> str:
    """
    Answers a question using a two-stage retrieval and reranking pipeline.
    """
    logging.info(f"\n🔍 Answering question: {question}")
    start_time = time.time()

    # --- Tahap 1: Retrieval ---
    logging.info(f"   1. Retrieving top {top_k_retrieval} candidates...")
    question_embedding = embedder_model.encode([question], convert_to_numpy=True)
    retrieval_hits = util.semantic_search(question_embedding, embeddings, top_k=top_k_retrieval)
    retrieval_hits = retrieval_hits[0] # Ambil hasil untuk query pertama

    if not retrieval_hits:
        logging.warning("   -> No candidates found in retrieval stage.")
        return "Maaf, tidak ada informasi awal yang relevan ditemukan."

    candidate_indices = [hit['corpus_id'] for hit in retrieval_hits]
    retrieval_scores = [hit['score'] for hit in retrieval_hits] # Skor awal (opsional)
    candidate_chunks = [chunks[i] for i in candidate_indices if 0 <= i < len(chunks)] # Ambil teks kandidat

    logging.info(f"   -> Found {len(candidate_chunks)} initial candidates (scores: {retrieval_scores[:5]}...).")

    # --- Tahap 2: Reranking ---
    if not candidate_chunks:
         logging.warning("   -> No valid candidate chunks to rerank.")
         return "Maaf, terjadi masalah saat mengambil kandidat relevan."

    logging.info(f"   2. Reranking {len(candidate_chunks)} candidates...")
    # Buat pasangan (pertanyaan, kandidat_chunk) untuk CrossEncoder
    rerank_pairs = [[question, chunk] for chunk in candidate_chunks]

    # Dapatkan skor dari CrossEncoder
    rerank_scores = cross_encoder_model.predict(rerank_pairs, show_progress_bar=False) # Jangan tampilkan progress bar di production/log

    # Gabungkan skor rerank dengan chunk asli
    reranked_results = list(zip(rerank_scores, candidate_chunks))

    # Urutkan berdasarkan skor rerank (tertinggi dulu)
    reranked_results.sort(key=lambda x: x[0], reverse=True)

    logging.info(f"   -> Reranked scores (top 5): {[f'{s:.4f}' for s, c in reranked_results[:5]]}...")

    # --- Tahap 3: Seleksi Jawaban Akhir ---
    logging.info(f"   3. Selecting top {top_n_final} final answers...")
    final_chunks = [chunk for score, chunk in reranked_results[:top_n_final]]

    if not final_chunks:
        logging.warning("   -> No chunks remained after reranking selection.")
        # Fallback: mungkin kembalikan hasil retrieval teratas jika reranking gagal total?
        # final_chunks = candidate_chunks[:top_n_final]
        return "Maaf, tidak dapat menentukan jawaban terbaik setelah penilaian ulang."

    # Gabungkan jawaban
    final_answer = "\n\n".join(final_chunks)
    end_time = time.time()
    logging.info(f"   -> Answer construction complete. Time taken: {end_time - start_time:.2f} seconds.")
    return final_answer

def post_process_answer(answer: str) -> str:
    """
    Membersihkan dan memformat jawaban mentah agar lebih mudah dibaca.
    - Tokenisasi kalimat
    - Hapus duplikasi kalimat
    - Format sebagai bullet list
    - Filter kalimat yang sangat pendek
    """
    print("   Post-processing answer...")
    # Ganti pemisah chunk asli (\n\n) dengan spasi agar sent_tokenize bekerja lebih baik
    # pada batas antar chunk.
    text_for_tokenize = answer.replace("\n\n", " ")
    sentences = sent_tokenize(text_for_tokenize)

    # Hapus duplikasi sambil menjaga urutan (membutuhkan Python 3.7+)
    unique_sentences = list(dict.fromkeys(sentences))

    # Filter kalimat pendek dan format sebagai bullet list
    bulleted_list = []
    min_sentence_length = 15 # Tingkatkan sedikit threshold panjang minimal
    for sentence in unique_sentences:
        cleaned_sentence = sentence.strip()
        # Filter tambahan: cek jika kalimat hanya berisi spasi atau non-alpha
        if len(cleaned_sentence) > min_sentence_length and re.search(r'[a-zA-Z]', cleaned_sentence):
             # Capitalize first letter of each sentence for consistency
             formatted_sentence = cleaned_sentence[0].upper() + cleaned_sentence[1:]
             bulleted_list.append(f"* {formatted_sentence}")

    if not bulleted_list:
         print("   -> No valid sentences remained after post-processing.")
         # Kembalikan jawaban asli jika post-processing menghapus semuanya
         # atau pesan default
         return answer if answer.strip() else "Tidak ada informasi detail yang dapat ditampilkan."

    final_output = "\n".join(bulleted_list)
    print(f"   -> Post-processing complete. Generated {len(bulleted_list)} bullets.")
    return final_output

# TESTING
Chatbot menampilkan mengambil dan menampilkan 3 chunk teks yang paling mirip dengan pertanyaan pengguna sebagai jawaban.

In [ ]:
# ===============================
# 10. TESTING CHATBOT (dengan Reranking)
# ===============================
# Pastikan variabel yang diperlukan sudah ada
if 'all_chunks' in locals() and 'chunk_embeddings' in locals() and 'embedder' in locals() and 'cross_encoder' in locals():

    logging.info("\n================ TESTING CHATBOT (with Reranking) ================")

    test_questions = [
        "Apakah Dana BOSP dapat digunakan untuk pengembangan sumber daya manusia?",
        "Untuk apa saja Dana BOS Kinerja dapat digunakan?",
        "Kapan laporan realisasi penggunaan Dana BOSP harus disampaikan?",
        "Sebutkan komponen penggunaan dana BOS Reguler",
        "Apa saja larangan penggunaan dana BOSP?", # Pertanyaan sulit sebelumnya
    ]

    # Parameter untuk pipeline
    RETRIEVAL_CANDIDATES = 15 # Ambil 15 kandidat awal
    FINAL_ANSWERS = 3         # Pilih 3 jawaban terbaik setelah rerank

    for question in test_questions:
        # Panggil fungsi baru dengan argumen reranking
        raw_answer = answer_question_with_rerank(
            question=question,
            chunks=all_chunks,
            embeddings=chunk_embeddings,
            embedder_model=embedder,
            cross_encoder_model=cross_encoder, # Berikan cross_encoder
            top_k_retrieval=RETRIEVAL_CANDIDATES,
            top_n_final=FINAL_ANSWERS
        )

        # Post-process jawaban
        processed_answer = post_process_answer(raw_answer)

        # Cetak hasil
        print(f"\n🔹 **Pertanyaan:** {question}")
        print(f"🔸 **Jawaban (Processed):**\n{processed_answer}\n")
        print("-" * 60) # Pemisah

    logging.info("================ END OF TEST ================")

else:
    logging.error("\nERROR: Cannot run chatbot test.")
    missing = [var for var in ['all_chunks', 'chunk_embeddings', 'embedder', 'cross_encoder'] if var not in locals()]
    logging.error(f"Required variables missing: {missing}")
    logging.error("Please ensure the loading/encoding steps (Section 6) have run successfully.")

   Post-processing answer...
   -> Post-processing complete. Generated 2 bullets.

🔹 **Pertanyaan:** Apakah Dana BOSP dapat digunakan untuk pengembangan sumber daya manusia?
🔸 **Jawaban (Processed):**
* Komponen Penggunaan Dana BOS Kinerja untuk Sekolah yang Melaksanakan Program Sekolah Penggerak yang ditetapkan sebagai pelaksana Program pengimbasan adalah pembinaan dan pengembangan transformasi kepada satuan pendidikan lain untuk melakukan peningkatan mutu dalam hal pengembangan sumber daya manusia, pembelajaran kurikulum merdeka, digitalisasi sekolah, dan perencanaan berbasis data yang dilaksanakan dalam bentuk kegiatan seperti: a. pelatihan; b. penguatan komunitas belajar; dan c pengembangan sumber daya manusia; b. pembelajaran kurikulum merdeka; c. digitalisasi sekolah; dan d. perencanaan berbasis data.
* (2a) Komponen penggunaan Dana BOS Kinerja bagi sekolah yang melaksanakan Program Sekolah Penggerak sebagaimana dimaksud pada ayat (1) huruf a yang ditetapkan sebagai pelaksana pro